In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *
from os.path import abspath

spark = SparkSession\
        .builder\
        .appName("pyspark-notebook")\
        .master("spark://spark-master:7077")\
        .config("spark.executor.memory", "512m")\
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
        .enableHiveSupport()\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

25/02/08 09:36:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 3. The Data

In [2]:
!apt install wget -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.20.1-1.1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [3]:
!wget "https://drive.google.com/uc?export=download&id=1wdjtBKoYRYaNVv3XUNNwcGbaNUSfok7U" -O uplit_data.zip


--2025-02-08 09:36:08--  https://drive.google.com/uc?export=download&id=1wdjtBKoYRYaNVv3XUNNwcGbaNUSfok7U
Resolving drive.google.com (drive.google.com)... 173.194.221.194, 2a00:1450:4010:c0a::c2, 2a00:1450:4010:c0a::c2
Connecting to drive.google.com (drive.google.com)|173.194.221.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1wdjtBKoYRYaNVv3XUNNwcGbaNUSfok7U&export=download [following]
--2025-02-08 09:36:09--  https://drive.usercontent.google.com/download?id=1wdjtBKoYRYaNVv3XUNNwcGbaNUSfok7U&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.14.132, 2a00:1450:4010:c0f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.14.132|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-02-08 09:36:10 ERROR 404: Not Found.



In [4]:
!unzip uplift_data.zip

unzip:  cannot find or open uplift_data.zip, uplift_data.zip.zip or uplift_data.zip.ZIP.


In [5]:
ls

pyspark.ipynb*  uplit_data.zip


In [6]:
data = spark.read.csv(path="datasets/data.csv", sep=",", header=True)

AnalysisException: Path does not exist: file:/opt/workspace/datasets/data.csv;

In [ ]:
data = spark.read.csv(path="clients.csv", sep=",", header=True)

In [ ]:
data.count()

In [ ]:
df1 = data.select(["column1", "column2", "column3"])

In [ ]:
df1 = data.select(["column1", "column2"])

In [ ]:
cols_description = df1.filter(df1['column1'] == 'Units')

In [ ]:
df1 = df1.join(other=cols_description, on=['column1'], how='left_anti')

In [ ]:
df1 = df1.\
               withColumnRenamed("column1", 'c1').\
               withColumnRenamed("column2", "c2").\
               withColumnRenamed("column3", "c3")

In [ ]:
df1.write.saveAsTable('hive_table')

In [ ]:
ndf= spark.sql("""
select * from hive_table
""")


In [ ]:
ndf.show(10)